In [2]:
import pandas as pd
import nltk
from nltk.util import ngrams
from collections import Counter

In [3]:
reviews = pd.read_csv('sushi_reviews.csv')

In [5]:
reviews

,Unnamed: 0,review_id,user_id,business_id,stars,useful,funny,cool,text,date,V10
0,1,pUycOfUwM8vqX7KjRRhUEA,59MxRhNVhU9MYndMkz0wtw,gebiRewfieSdtt17PTW6Zg,3,0,0,0,Had a party of 6 here for hibachi. Our waitres...,2016-07-25 07:31:06,party 6 hibachi waitress brought separa...
1,2,eCiWBf1CJ0Zdv1uVarEhhw,OhECKhQEexFypOMY6kypRw,vC2qm1y3Au5czBtbhc-DNw,4,0,0,0,"Yes, this is the only sushi place in town. How...",2013-09-04 03:48:20,yes sushi place town however great you...
2,3,lUUhg8ltDsUZ9h0xnwY4Dg,RreNy--tOmXMl1en0wiBOg,cPepkJeRMtHapc_b2Oe_dw,4,1,0,1,I was really between 3 and 4 stars for this on...,2018-07-17 03:30:07,really 3 4 stars one love 96th street ...
3,4,ymhbOMW63B_vGaRFR3XT0A,yZdAhQ_KmKuCZmbBy9YDQg,5Ce3lZksYVkCbrihqylVHQ,5,0,0,0,I just started going to Sushi Nara this month ...,2014-07-25 17:56:26,just started going sushi nara month opened...
4,5,RMho6HMpdec1YgIypwWQrQ,SNngOVGTkD34B3FAFnMv5A,ab3pRv-b0o-BwMK2jVbH3Q,5,0,0,0,My husband and I come here often. We love the ...,2018-08-24 00:52:13,husband come often love food always fr...
...,...,...,...,...,...,...,...,...,...,...,...
215550,215551,3meLluIIU_TFZS1IP9-DcA,6He7CwCnjxeCN9Cs-X8i7g,3FPi2yKCIh0Hh3iBg8faYA,1,5,1,0,"I used to like coming to this restaurant, but ...",2019-02-28 02:10:45,used like coming restaurant last four ti...
215551,215552,2ANk7BzQeUxvC3-sPmkd0w,3oj2pnNtv_eTD-42WGdRFQ,zSSMmKQujl731_voEHStmg,5,0,0,0,I am very happy to have this restaurant in my ...,2015-02-01 21:41:28,happy restaurant neighborhood discov...
215552,215553,FeghE6yk5hmgGFYYqkr5-Q,HiRGRdbSbR7btUmWuZ4_IQ,-5UQGwUZWUhPi5pJlcwEYA,4,2,0,0,"The box lunch is my favorite take out lunch, a...",2016-11-05 20:28:20,box lunch favorite take lunch reasonab...
215553,215554,Wrt6pZizQzw-ZTKrvMwrGw,ua6QuBe6mar6pDrhHETzJQ,Bk_1vsPtOtO0bojfQZQIOw,2,3,3,2,"Ok, I have to say, after living on this block ...",2008-12-15 23:18:27,ok say living block 4 months passing ...


In [18]:
%%time
bag = reviews['V10'].tolist()
bag = ' '.join(str(e) for e in bag)

Wall time: 17 ms


In [37]:
%%time
bigram = list(ngrams(nltk.word_tokenize(bag), 2))  #gen bigrams from bag
temp = dict(Counter(bigram))

df = pd.DataFrame(list(temp.items()))

df.columns = ['Bi-Gram', 'Frequency']
df = df.loc[df.Frequency > 5] #filter more frequent grams 5 mentions or more
df = df.sort_values(by=['Frequency'],ascending=False)
df.to_csv('bigrams_all_reviews.csv')

Wall time: 53.6 s


In [38]:
%%time
trigram = list(ngrams(nltk.word_tokenize(bag), 3))  #gen trigrams from bag
temp = dict(Counter(trigram))

df = pd.DataFrame(list(temp.items()))

df.columns = ['Tri-Gram', 'Frequency']
df = df.loc[df.Frequency > 5] #filter more frequent grams 5 mentions or more
df = df.sort_values(by=['Frequency'],ascending=False)
df.to_csv('trigrams_all_reviews.csv')

Wall time: 57.4 s


In [51]:
df_uni = pd.read_csv('unigrams_all_reviews.csv')
df_bi = pd.read_csv('bigrams_all_reviews.csv')
df_tri = pd.read_csv('trigrams_all_reviews.csv')


In [50]:
all_grams = pd.concat([df_uni, df_bi, df_tri], axis=1)
all_grams = all_grams.drop(['Unnamed: 0'], axis=1)

all_grams.to_csv('all_grams_all_reviews.csv', index = False)

# Filter Reviews by star and make n-gram tables 


In [62]:
%%time
#first loop for filter new dataset
for i in range(1,6): #stars
    current_star = f'{i}-Star'
    temp_df = reviews.loc[reviews.stars == i]

    bag = temp_df['V10'].tolist()
    bag = ' '.join(str(e) for e in bag)

    #second loop for 1,2,3 grams
    for j in range(1,4):
        current_gram = f'{j}-Gram'
        grams = list(ngrams(nltk.word_tokenize(bag), j))  #gen j-Grams
        temp = dict(Counter(grams))
        if j == 1:
            gram_df = pd.DataFrame(list(temp.items()))
            gram_df.columns = [current_gram, 'Frequency']
            gram_df = gram_df.loc[gram_df.Frequency > 5] #filter more frequent grams 5 mentions or more
            gram_df = gram_df.sort_values(by=['Frequency'],ascending=False)
    
        else:
            temp = pd.DataFrame(list(temp.items()))
            temp.columns = [current_gram, 'Frequency']
            temp = temp.loc[temp.Frequency > 5] #filter more frequent grams 5 mentions or more
            temp = temp.sort_values(by=['Frequency'],ascending=False)
            gram_df = pd.concat([gram_df, temp], axis=1)

    gram_df.to_csv(f'{current_star}-grams.csv')

Wall time: 2min 38s
